# Příklad použití nástroje AutoGen


## Importujte potřebné balíčky


In [ ]:
import os
import json

import requests
from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from typing import Any, Callable, Set, Dict, List, Optional

## Vytvoření klienta

V tomto příkladu použijeme [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) pro přístup k LLM.

`model` je definován jako `gpt-4o-mini`. Zkuste změnit model na jiný dostupný na tržišti GitHub Models a podívejte se na různé výsledky.

Jako rychlý test jednoduše spustíme jednoduchý dotaz - `What is the capital of France`.


In [ ]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## Definování funkcí

V tomto příkladu poskytneme agentovi přístup k nástroji, který je funkcí obsahující seznam dostupných dovolenkových destinací a jejich dostupnost.

Můžete si představit, že by se mohlo jednat o situaci, kdy má cestovní agent přístup například k cestovní databázi.

Jak budete procházet tímto příkladem, neváhejte zkusit definovat nové funkce a nástroje, které může agent volat.


In [ ]:
from typing import Dict, List, Optional


def vacation_destinations(city: str) -> tuple[str, str]:
    """
    Checks if a specific vacation destination is available
    
    Args:
        city (str): Name of the city to check
        
    Returns:
        tuple: Contains city name and availability status ('Available' or 'Unavailable')
    """
    destinations = {
        "Barcelona": "Available",
        "Tokyo": "Unavailable",
        "Cape Town": "Available",
        "Vancouver": "Available",
        "Dubai": "Unavailable",
    }

    if city in destinations:
        return city, destinations[city]
    else:
        return city, "City not found"

# Example usage:
# city, status = vacation_destinations("Barcelona")
# print(f"How about visiting {city}? It's currently {status} there!")

## Definování nástroje funkce  
Aby agent mohl použít `vacation_destinations` jako `FunctionTool`, musíme jej takto definovat.  

Poskytneme také popis nástroje, který agentovi pomůže pochopit, k čemu je tento nástroj určen ve vztahu k úkolu, který uživatel požaduje.


In [ ]:
get_vacations = FunctionTool(
    vacation_destinations, description="Search for vacation destinations and if they are available or not."
)

## Definování agenta

Nyní můžeme vytvořit agenta v níže uvedeném kódu. Definujeme `system_message`, aby agentovi poskytl instrukce, jak pomoci uživatelům najít destinace pro dovolenou.

Také nastavíme parametr `reflect_on_tool_use` na hodnotu true. To umožňuje LLM vzít odpověď z volání nástroje a poslat ji pomocí přirozeného jazyka.

Můžete nastavit parametr na hodnotu false, abyste viděli rozdíl.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[get_vacations],
    system_message="You are a travel agent that helps users find vacation destinations.",
    reflect_on_tool_use=True,
)

## Spuštění agenta

Nyní můžeme spustit agenta s počáteční zprávou od uživatele, který žádá o výlet do Tokia.

Můžete změnit tuto cílovou destinaci a zjistit, jak agent reaguje na dostupnost města.


In [ ]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="I would like to take a trip to Tokyo", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()


---

**Prohlášení**:  
Tento dokument byl přeložen pomocí služby pro automatický překlad [Co-op Translator](https://github.com/Azure/co-op-translator). Ačkoli se snažíme o přesnost, mějte na paměti, že automatické překlady mohou obsahovat chyby nebo nepřesnosti. Původní dokument v jeho původním jazyce by měl být považován za autoritativní zdroj. Pro důležité informace doporučujeme profesionální lidský překlad. Neodpovídáme za žádná nedorozumění nebo nesprávné interpretace vyplývající z použití tohoto překladu.
